---
#1 Introducción
En el siguiente ejercicio se realiza la aproximación del número pi por el método de Montecarlo.

El algoritmo consiste en la generación de puntos aleatorios en un determinado cuadrante. Si esos puntos, pertenecen al círculo que abarca esa área, se van contando. Finalmente, se utilizan en la siguiente fórmula para aproximarlo:

${pi}={4}*\frac{PuntosDentroDelCirculo}{Puntos Totales}$

La idea no es solo desarrollar el algoritmo en sí, sino ver su comparativa en tiempos de procesamiento respecto a las diferentes ejecuciones (Secuencial y en paralelo). Para su implementación se utilizó la interfaz OpenMP que nos permite paralelizar procesos. 

El código está desarrollado en C, es compilado por el compilador gcc, dicha compilación es ejecutada desde Python. Para finalizar, se realiza la aproximación, pero desde Python, se grafican los puntos obtenidos y su círculo correspondiente y se muestran funcionalidades tales como descargar el gráfico resultante, descargar un excel con los puntos obtenidos o crear una hoja de cálculo en Google Sheets.

---
#2 Armado del ambiente
Instala en el cuaderno el módulo gspread de Python.


In [ ]:
!pip install --upgrade gspread

---
#3 Desarrollo
Ejecución del algoritmo.


In [ ]:
code = """
#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/time.h>

#define SEED time(NULL)
#define RANDOM_MOD 0x80000000

//Declaración de funciones
int calcularCantidadPuntosEnCirculo(const int particion);
int random_int(void);
double random_double(double rango);

int main(int argc, char* argv[])
{
    //Declaración de variables
    int i, contador, cantidad_numeros, particiones, particion;
    double x, y, z, pi, tiempo;
    struct timeval tiempo_inicio, tiempo_fin, tiempo_total_inicio, tiempo_total_fin;

    //Valida los parámetros
    if(argc != 2)
    {
        printf("Error de parámetros.");
        return(-1);
    }

    //Inicia tiempo de ejecución total
    gettimeofday(&tiempo_total_inicio, NULL);

    //Genera la semilla para números aleatorios
    srand(SEED);

    //Obtiene el argumento y lo guarda en la variable cantidad_numeros
    cantidad_numeros = atoi(argv[1]);
    
    //Valida los parámetros
    if(cantidad_numeros <= 0)
    {
        printf("La cantidad de números debe ser mayor a cero.");
        return(-1);
    }

    //////Montecarlo Secuencial//////

    //Inicia tiempo de Montecarlo secuencial
    gettimeofday(&tiempo_inicio, NULL);

    //Inicializa el contador
    contador = 0;

    //Loop Montecarlo secuencial
    for(i = 0; i < cantidad_numeros; ++i)
    {
        //Generan un punto aleatorio
        x = (double)rand() / RAND_MAX;
        y = (double)rand() / RAND_MAX;

        z = x * x + y * y;

        //Incrementa si pertenece al círculo
        if( z <= 1 )
            contador++;
    }

    //Fórmula Montecarlo
    pi = (double) contador / cantidad_numeros * 4;

    //Finaliza tiempo de Montecarlo secuencial
    gettimeofday(&tiempo_fin, NULL);

    //Calcula tiempo de Montecarlo secuencial
    tiempo = tiempo_fin.tv_sec + tiempo_fin.tv_usec / 1e6 -
                 tiempo_inicio.tv_sec - tiempo_inicio.tv_usec / 1e6;

    //Muestra datos de Montecarlo secuencial
    printf("\\n<------------------------------------------------------->");
    printf("\\nMONTECARLO SECUENCIAL");
    printf("\\nTiempo de cálculo PI secuencial = %f [seg]", tiempo);
    printf("\\nAproximación de secuencial = %g", pi);

    //////Montecarlo OpenMP//////
    
    //Inicia tiempo de Montecarlo OpenMP
    gettimeofday(&tiempo_inicio, NULL);

    //Inicializo particiones
    particiones = 8;
    particion = cantidad_numeros / particiones;
    contador = 0;

    //Inicia procesamiento en paralelo 
    #pragma omp parallel for shared(particiones, particion) reduction(+:contador)
    for (int i = 0; i < particiones; i++)
    {
        contador += calcularCantidadPuntosEnCirculo(particion);
    }

    //Fórmula Montecarlo
    pi = (double) contador / cantidad_numeros * 4;

    //Finaliza tiempo de Montecarlo OpenMP
    gettimeofday(&tiempo_fin, NULL);

    //Calcula tiempo de Montecarlo OpenMP
    tiempo = tiempo_fin.tv_sec + tiempo_fin.tv_usec / 1e6 -
                 tiempo_inicio.tv_sec - tiempo_inicio.tv_usec / 1e6;

    //Muestra datos de Montecarlo OpenMP     
    printf("\\n<------------------------------------------------------->");
    printf("\\nMONTECARLO OPENMP");
    printf("\\nTiempo de cálculo PI OpenMP = %f [seg]", tiempo);
    printf("\\nAproximación de PI OpenMP = %g", pi);

    //Finaliza tiempo de ejecución total
    gettimeofday(&tiempo_total_fin, NULL);

    //Calcula tiempo de ejecución total
    tiempo = tiempo_total_fin.tv_sec + tiempo_total_fin.tv_usec / 1e6 -
                 tiempo_total_inicio.tv_sec - tiempo_total_inicio.tv_usec / 1e6;

    //Muestra datos de ejecución
    printf("\\n<------------------------------------------------------->");
    printf("\\nDATOS DE EJECUCIÓN");
    printf("\\nTiempo de ejecución total = %f [seg]", tiempo);
    printf("\\nCantidad de números = %d", cantidad_numeros);

    return(0);
}


//Función Montecarlo OpenMP
int calcularCantidadPuntosEnCirculo(const int particion)
{
    //Definición de variables
    int i, contador, s;
    double x,y,z;

    //Inicializa el contador
    contador = 0;

    //Loop Montecarlo OpenMP
    for (s = 0; s != particion; s++)
    {
        //Generan un punto aleatorio
        x = random_double(1.0);
        y = random_double(1.0);

        z = x * x + y * y;

        //Incrementa si pertenece al círculo
        if( z <= 1 )
            contador++;
    }
    return contador;
}

//Función para cálculo de número aleatorio de tipo int
int random_int(void)
{
    int estado;
    return (estado = (estado * 1103515245 + 12345) & 0x7fffffff);
}

//Función para cálculo de número aleatorio de tipo double
double random_double(double rango)
{
    return ((double)random_int()) / (((double)RANDOM_MOD)/rango);
}
"""
try:
  text_file = open("montecarlo.cpp", "w")
  text_file.write(code)
  text_file.close()
except:
  print("Error de archivo.")

---
## Compilación 3.1
Recibe el programa fuente en lenguaje C y genera un programa ejecutable binario en el lenguaje de la máquina en la que se está corriendo.


In [ ]:
try:
  !gcc -o montecarlo -fopenmp montecarlo.cpp
except:
  print("Error de compilación.")

---
##Ejecución 3.2
Inicialmente se define la cantidad de hilos que utilizará OpenMP, luego, se ejecuta el código compilado el cual en su llamado recibe la cantidad de números que utilizará el método de Montecarlo para la aproximación del número pi.


In [ ]:
#Cantidad de hilos
%env OMP_NUM_THREADS = 8

#Ejecución
!./montecarlo 50000

En las primeras dos secciones de la salida se puede apreciar la diferencia de tiempos de procesamiento entre la ejecución secuencial y la ejecución con OpenMP. En la última sección, se detallan los datos de ejecución (Tiempo de ejecución total y la cantidad de números con la que trabajó el método de Montecarlo).

---
##3.3 ¡Montecarlo en Python!
El siguiente fragmento de código aproxima el número pi bajo el método de Montecarlo pero desde el punto de vista de Python. Además, en él se trabaja con distintas librerías externas con el objetivo de explorar la capacidad de Colab. Estas librerías son tales como: matplotlib.pyplot (Para realizar el gráfico que resulta del método Montecarlo en sí), pandas (Para la confexión de un excel con los puntos obtenidos del algoritmo), gspread (Para crear una hoja de cálculo en Google Sheets con los puntos obtenidos del algoritmo). También, se anexan distintas formas de interacción con el cuaderno de Colab, como es la descarga de archivos, tanto como del gráfico obtenido como del excel generado.

In [ ]:
#@title ### 3.3.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la cantidad de números:
cantidad_numeros =   1000#@param {type: "number"}
#@markdown ---

#Valida parámetro
if cantidad_N <= 0:
  print('La cantidad de elemtentos debe ser mayor a cero.')
else:
  import numpy as np
  import matplotlib.pyplot as plt
  from google.colab import files
  from google.colab import auth
  import pandas as pd
  import gspread
  from oauth2client.client import GoogleCredentials

  #Creamos un punto aleatorio entre -1 y 1
  def punto():
      coordenadas=[np.random.uniform(-1,1), np.random.uniform(-1,1)]
      return coordenadas[0]**2+coordenadas[1]**2<1, coordenadas

  #Método Montecarlo
  def montecarlo(cantidad_numeros):
      #Inizializamos contador
      contador=0

      #Definición de vectores de coordenadas
      Xdentro=[]
      Ydentro=[]
      Xfuera=[]
      Yfuera=[]

      #Loop Montecarlo
      for i in range(cantidad_numeros):
          estado, coordenadas = punto()
          if estado:
              contador+=1
              Xdentro.append(coordenadas[0])
              Ydentro.append(coordenadas[1])
          else:
              Xfuera.append(coordenadas[0])
              Yfuera.append(coordenadas[1])

      #Fórmula Montecarlo y puntos       
      return 4*contador/cantidad_numeros, Xdentro, Ydentro, Xfuera, Yfuera

  #Calcula PI
  pi=montecarlo(cantidad_numeros)

  #Define alto y ancho del gráfico en pulgadas
  fig=plt.figure(figsize=(7,7))

  #Crea una grilla para el gráfico 1x1
  ax=fig.add_subplot(1, 1, 1)

  #Configura el escalado de los datos para el trazado en el gráfico
  ax.set_aspect('equal')

  #Grafica el circulo de fondo
  circulo = plt.Circle((0, 0), 1, color='0.95', fill=True, zorder=1)
  ax.add_artist(circulo)

  #Grafica los puntos
  ax.scatter(pi[1],pi[2], s=0.2, color='g', marker="o", zorder=2)
  ax.scatter(pi[3],pi[4], s=0.2, color='r', marker="o", zorder=3)

  #Define los límites visibles del gráfico en X e Y
  plt.xlim(-1, 1)
  plt.ylim(-1, 1)

  #Define el título para el gráfico
  plt.title('Aproximación de π - Método Montecarlo')

  try:
    #Guarda la imagen en el contexto de ejecución del cuaderno Colab
    plt.savefig('montecarlo.png')
  except:
    print("No se pudo guardar la imagen en el contexto de ejecución del cuaderno de Colab.")

  #Almacena los puntos en un vector
  puntos = [pi[1],pi[2],pi[3],pi[4]]

  #Carga los puntos en un dataframe
  df = pd.DataFrame(puntos).T

  try:
    #Guarda el excel en el contexto de ejecución del cuaderno Colab
    df.to_excel(excel_writer = 'montecarlo.xlsx')
  except:
    print("No se pudo guardar el excel en el contexto de ejecución del cuaderno de Colab.")

  #Muestra el gráfico
  plt.show()


###3.3.2 Funcionalidades especiales
Se pueden ejecutar las siguientes funcionalidades realizadas con intenciones explorativas sobre la capacidad de Colab.

####3.3.2.1 Descarga de gráfico
##### Se puede descargar el gráfico obtenido en formato PNG ejecutando el siguiente fragmento de código:

In [ ]:
try:
  #Descarga la imagen
  files.download('montecarlo.png')
except:
  print("Error al descargar la imagen.")


####3.3.2.2 Descarga de excel
#####Se puede descargar un excel con los puntos obtenidos ejecutando el siguiente fragmento de código:

In [ ]:
try:
  #Descarga el excel
  files.download('montecarlo.xlsx')
except:
  print("Error al descargar el excel.")

####3.3.2.3 Crear hoja de cálculo en Google Sheets
#####Se puede crear una hoja de cálculo de Google Sheets ejecutando el siguiente fragmento de código:

In [ ]:
try:
  #Solicita la autorización de permisos de Google Sheets
  auth.authenticate_user()

  #Linkea el permiso
  gc = gspread.authorize(GoogleCredentials.get_application_default())
except:
  print("Error de autenticación.")
  
try:
  #Crea la hoja de cálculo
  hc = gc.create('montecarlo')
except:
  print("Error al crear la hoja de cálculo.")

try:
  #Abre la hoja de cálculo en su primer página
  hoja_de_calculo = gc.open('montecarlo').sheet1
except:
  print("Error al abrir la hoja de cálculo.")

try:
  #Inserta los puntos en la hoja de cálculo
  hoja_de_calculo.update(puntos)
except:
  print("Error al modificar la hoja de cálculo.")

---
#4 Tabla de pasos

Entorno     |Función               |Detalle
------------|----------------------|-------------------------
Colab       |pip install gspread   |Instala en el cuaderno el módulo gspread de Python.
C           |include               |Importan las librerías necesarias.
C           |define                |Definen constantes.
C           |if                    |Valida los parámetros.
C           |gettimeofday          |Inicia tiempo de ejecución total.
C           |srand                 |Genera la semilla para números aleatorios.
C           |atoi                  |Transforma el parámetro de string a int.
C           |if                    |Valida los parámetros.
C           |gettimeofday          |Inicia tiempo de Montecarlo secuencial.
C           |for                   |Loop de Montecarlo secuencial.
C           |rand                  |Genera un número aleatorio para X e Y.
C           |gettimeofday          |Finaliza tiempo de Montecarlo secuencial.
C           |printf                |Muestra los datos de Montecarlo secuencial.
C           |gettimeofday          |Inicia tiempo de Montecarlo OpenMP.
C           |pragma                |Paralelización.
C           |shared                |Especifica variables compartidas entre todos los procesos.
C           |reduction             |Especifica la operación de reducción y la variable en donde se almacena.
C           |for                   |Loop de Montecarlo OpenMP.
C           |random_double         |Genera un número aleatorio para X e Y.
C           |gettimeofday          |Finaliza tiempo de Montecarlo OpenMP.
C           |printf                |Muestra los datos de Montecarlo OpenMP.
C           |gettimeofday          |Finaliza tiempo de ejecución total.
C           |printf                |Muestra los datos de ejecución.
Colab       |open                  |Abre el archivo en modo de escritura.
Colab       |write                 |Escribe en el archivo el código C.
Colab       |close                 |Cierra el archivo.
Colab       |gcc                   |Recibe el programa fuente en lenguaje C y genera un programa ejecutable binario en el lenguaje de la máquina en la que se está corriendo. Se le indica que se utiliza la interfaz OpenMP.
Colab       |%env OMP_NUM_THREADS  |Define la cantidad de hilos que dispondrá OpenMP por medio de la variable de entorno en cuestión.
Colab       |montecarlo            |Ejecuta el código.
Colab       |if                    |Valida parámetro.
Colab       |random.uniform        |Devuelve un número aleatorio entre -1 y 1.
Colab       |for                   |Loop Montecarlo.
Colab       |append                |Agrega el valor al vector del punto.
Colab       |figure                |Define alto y ancho del gráfico en pulgadas.
Colab       |add_subplot           |Crea una grilla para el gráfico 1x1.
Colab       |set_aspect            |Configura el escalado de los datos para el trazado en el gráfico.
Colab       |Circle                |Grafica el círculo de fondo.
Colab       |add_artist            |Agrega un Artist al gráfico y lo retorna.
Colab       |scatter               |Grafica los puntos.
Colab       |xlim / ylim           |Define los límites visibles del gráfico en X e Y.
Colab       |title                 |Define el título para el gráfico.
Colab       |savefig               |Guarda la imagen en el contexto de ejecución del cuaderno de Colab.
Colab       |DataFrame             |Carga los puntos en un dataframe.
Colab       |to_excel              |Guarda el excel en el contexto de ejecución del cuaderno de Colab.
Colab       |show                  |Muestra el gráfico.
Colab       |download              |Descarga la imagen.
Colab       |download              |Descarga el excel.
Colab       |authenticate_user     |Solicita la autorización de permisos de Google Sheets.
Colab       |authorize             |Linkea el permiso.
Colab       |create                |Crea la hoja de cálculo.
Colab       |open ... sheet1       |Abre la hoja de cálculo en su primer página.
Colab       |update                |Inserta los puntos en la hoja de cálculo.














---
#5 Conclusiones

Como una conclusión general y empírica se puede observar que los datos, tras cierta cantidad de ejecuciones, arrojan que el procesamiento en paralelo es muchísimo más eficiente para cuando se requiere trabajar con una gran cantidad de números.

Por ejemplo:

Ejecución de algoritmo secuencial: 
27.181815 [seg] con 1000000000 números.

Ejecución del algoritmo OpenMP: 
14.380366 [seg] con 1000000000 números.

Se puede ver claramente la diferencia de tiempos entre cada ejecución.

Se puede apreciar que si la cantidad de números es poca pierde eficiencia el algoritmo OpenMP y pasa a ser mejor el algoritmo secuencial.

Ejecución de algoritmo secuencial: 
0.000313 [seg] con 10000 números.

Ejecución del algoritmo OpenMP: 
0.000548 [seg] con 10000 números.

Esto se debe a que como el hilo hace el context switch manejando su estructura de control, se pierde más tiempo que trabajar el intercambio de manera secuencial.

Respecto a lo que es el método de Montecarlo en sí... Es un método que es genial para determinar áreas extrañas, sin embargo, se requiere una ENORME cantidad de números aleatorios para que se logre una aproximación a pi acertada, existen otros métodos de aproximación mucho más eficientes que éste.

Respecto a la implementación realizada, la precisión del procesamiento secuencial es un poco mejor que la precisión del procesamiento en paralelo.

**Comentario personal:**

Al momento de generar números aleatorios, con la funcion rand(), dentro del procesamiento en paralelo, la ejecución con OpenMP tardaba 100 veces más que la ejecución secuencial. Tuve que implementar una función para obtener un número seudo aleatorio distinta a ella. Se ve que tiene que ver con que no es thread safe rand(), entonces genera una race condition haciendo realmente ineficiente al procesamiento en paralelo. Luego de implementar mi función el algoritmo mejoró considerablemente y se empezó a comportar como lo esperado.

---
# 6 Bibliografía

[1] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[2] Colab and Google Sheets — Surprisingly Powerful Combination for Data Science (Part 1): [Medium](https://medium.com/analytics-vidhya/colab-and-google-sheets-surprisingly-powerful-combination-for-data-science-part-1-bbbb11cbd8e)

[3] Matplotlib.pyplot: [matplotlib](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot)

[4] El método Montecarlo: [Geogebra](https://www.geogebra.org/m/cF7RwK3H)

[5] Snippets: Sheets - Colaboratory: [Página Colab](https://colab.research.google.com/notebooks/snippets/sheets.ipynb)

[6] Charts in Colaboratory - Colaboratory: [Página Colab](https://colab.research.google.com/notebooks/charts.ipynb)

[7] Examples of gspread Usage: [gspread](https://gspread.readthedocs.io/en/latest/user-guide.html)

[8] Pandas documentation: [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)

[9] How to compile and run an OpenMP program: [Dartmouth](https://www.dartmouth.edu/~rc/classes/intro_openmp/compile_run.html)